# Data Preparation

In [1]:
import pandas as pd
import requests
import json
import time

## Coordinate Conversion

In [2]:
# set up the access token
with open("config.json") as config_file:
    config = json.load(config_file)
    access_token = config["onemaps_access_token"]

# function to convert x y to lat long
def convert_xy_to_lat_long(x, y, access_token):
    """
    Convert x and y coordinates to latitude and longitude 
    (EPSG:3414 to EPSG:4326)
    """
    url = f"https://www.onemap.gov.sg/api/common/convert/3414to4326?X={x}&Y={y}"
    headers = {"Authorization": f"{access_token}"}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [12]:
# Test response
convert_xy_to_lat_long(30342.66, 33560.57, access_token)

{'latitude': 1.3197843040957071, 'longitude': 103.85436859833848}

### HDB carpark data

In [3]:
#HDB carpark information
hdb_carpark_df = pd.read_csv("data/HDBCarparkInformation.csv")
hdb_carpark_df.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N
4,AK31,BLK 302/348 ANG MO KIO STREET 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N


In [10]:
hdb_carpark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   car_park_no             2226 non-null   object 
 1   address                 2226 non-null   object 
 2   x_coord                 2226 non-null   float64
 3   y_coord                 2226 non-null   float64
 4   car_park_type           2226 non-null   object 
 5   type_of_parking_system  2226 non-null   object 
 6   short_term_parking      2226 non-null   object 
 7   free_parking            2226 non-null   object 
 8   night_parking           2226 non-null   object 
 9   car_park_decks          2226 non-null   int64  
 10  gantry_height           2226 non-null   float64
 11  car_park_basement       2226 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 208.8+ KB


In [4]:
# iterate through the dataframe (x_coord and y_coord) to append new lat long columns
lat = []
long = []
for index, row in hdb_carpark_df.iterrows():
    x = row["x_coord"]
    y = row["y_coord"]
    try:
        response = convert_xy_to_lat_long(x, y, access_token)
        lat.append(response["latitude"])
        long.append(response["longitude"])
        print(f"Converting row {index} from x: {x}, y: {y} to lat: {response['latitude']}, long: {response['longitude']}")
        
        # set buffer affter 240 requests as The OneMap API has a rate limit of 250 requests per minute
        if index % 240 == 0 and index != 0:
            print("Buffering...")
            time.sleep(60)

    except Exception as e:
        print(f"Error encountered {e}. \n Retrying after 5 seconds...")
        time.sleep(10)  # Wait for 10 seconds before retrying
        # Retry the conversion
        response = convert_xy_to_lat_long(x, y, access_token)
        lat.append(response["latitude"])
        long.append(response["longitude"])
        print(f"Converting row {index} from x: {x}, y: {y} to lat: {response['latitude']}, long: {response['longitude']}")

hdb_carpark_df["latitude"] = lat
hdb_carpark_df["longitude"] = long

Converting row 0 from x: 30314.7936, y: 31490.4942 to lat: 1.3010632720874935, long: 103.85411804993093
Converting row 1 from x: 33758.4143, y: 33695.5198 to lat: 1.3210042901059578, long: 103.88506094761526
Converting row 2 from x: 29257.7203, y: 34500.3599 to lat: 1.3282834951094038, long: 103.84461988914597
Converting row 3 from x: 28185.4359, y: 39012.6664 to lat: 1.3690912161435573, long: 103.83498485034086
Converting row 4 from x: 29482.029, y: 38684.1754 to lat: 1.3661204243173262, long: 103.84663563161604
Converting row 5 from x: 29889.3457, y: 39382.8134 to lat: 1.3724386334422431, long: 103.85029569742512
Converting row 6 from x: 31397.2241, y: 39851.6191 to lat: 1.3766782086821612, long: 103.86384511310713
Converting row 7 from x: 29674.8184, y: 40616.875 to lat: 1.3835990576551414, long: 103.84836808665371
Converting row 8 from x: 28777.0707, y: 38973.9528 to lat: 1.368741094555009, long: 103.84030110265232
Converting row 9 from x: 28267.0582, y: 39151.8344 to lat: 1.370349

In [5]:
#save as csv
hdb_carpark_df.to_csv("data/HDBCarparkInformation_latlong.csv", index=False)

In [6]:
hdb_carpark_df.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement,latitude,longitude
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y,1.301063,103.854118
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N,1.321004,103.885061
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N,1.328283,103.844620
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N,1.369091,103.834985
4,AK31,BLK 302/348 ANG MO KIO STREET 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N,1.366120,103.846636


### Amenities Data

In [26]:
#Read HDB Property Information
df = pd.read_csv('data/HDBPropertyInformation.csv')
df

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12872,998A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,33,50,0,0,0,0,0,0,0,0
12873,998B,BUANGKOK CRES,17,2018,Y,N,N,N,N,N,...,31,47,0,0,0,0,0,0,0,0
12874,999,BUANGKOK CRES,2,2018,N,N,N,Y,N,Y,...,0,0,0,0,0,0,0,0,0,0
12875,999A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,0,0,0,0,0,0,136,170,0,0


In [27]:
df["Address"] = df["blk_no"] + " " + df["street"]
addresslist = list(df["Address"])

In [28]:
def getcoordinates(address):
    req = requests.get(
        "https://www.onemap.gov.sg/api/common/elastic/search?searchVal="
        + address
        + "&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    )
    resultsdict = eval(req.text)
    if len(resultsdict["results"]) > 0:
        return (
            resultsdict["results"][0]["LATITUDE"],
            resultsdict["results"][0]["LONGITUDE"],
        )
    else:
        pass

In [40]:
lat = []
long = []
len_addr_list = len(addresslist)
for index, address in enumerate(addresslist):
    try:
        if len(getcoordinates(address)) > 0:
            lat_value, long_value = getcoordinates(address)
            lat.append(lat_value)
            long.append(long_value)
            print(f"Converting row {index} out of {len_addr_list} from {address} to {float(lat_value):.4f}, {float(long_value):.4f}")

            # set buffer affter 240 requests as The OneMap API has a rate limit of 250 requests per minute
            if index % 240 == 0 and index != 0:
                print("Buffering...")
                time.sleep(60)

    except Exception as e:
        print("Failed to extract row", index, "out of", len(addresslist), "addresses")
        print(f"Error {e}. Retrying after 10 seconds...")
        time.sleep(10)
        if getcoordinates(address) != None:
            lat_value, long_value = getcoordinates(address)
            lat.append(lat_value)
            long.append(long_value)
            print(f"Converting row {index} out of {len_addr_list} from {address} to {float(lat_value):.4f}, {float(long_value):.4f}")
        else:
            lat.append(None)
            long.append(None)
            print(f"Appended none to row {index} out of {len_addr_list} addresses")


df["latitude"] = lat
df["longitude"] = long

Converting row 0 out of 12877 from 1 BEACH RD to 1.3037, 103.8645
Converting row 1 out of 12877 from 1 BEDOK STH AVE 1 to 1.3209, 103.9337
Converting row 2 out of 12877 from 1 CANTONMENT RD to 1.2755, 103.8414
Converting row 3 out of 12877 from 1 CHAI CHEE RD to 1.3280, 103.9227
Converting row 4 out of 12877 from 1 CHANGI VILLAGE RD to 1.3885, 103.9878
Converting row 5 out of 12877 from 1 DELTA AVE to 1.2921, 103.8286
Converting row 6 out of 12877 from 1 DOVER RD to 1.3025, 103.7833
Converting row 7 out of 12877 from 1 EUNOS CRES to 1.3207, 103.9025
Converting row 8 out of 12877 from 1 EVERTON PK to 1.2755, 103.8414
Converting row 9 out of 12877 from 1 GEYLANG SERAI to 1.3167, 103.8981
Converting row 10 out of 12877 from 1 GHIM MOH RD to 1.3129, 103.7869
Converting row 11 out of 12877 from 1 HAIG RD to 1.3113, 103.8977
Converting row 12 out of 12877 from 1 HOLLAND CL to 1.3092, 103.7967
Converting row 13 out of 12877 from 1 HOUGANG AVE 3 to 1.3641, 103.8930
Converting row 14 out of 128

In [44]:
#save df
df.to_csv("data/HDBPropertyInformation_latlong.csv", index=False)